In [40]:
import pandas as pd
from unidecode import unidecode
import re
import ast
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [41]:
import warnings

#Suppress all warnings
warnings.filterwarnings("ignore")


## DF JEANS:

In [42]:
# Remove accents with unidecode
def remove_accents(text):
    if isinstance(text, str):
        return unidecode(text)
    return text

In [43]:
df1 = pd.read_csv("/Users/admin/Desktop/IRONHACK/8.WEEK/final_project/notebooks/datos/jeans/df7.csv")
df2 = pd.read_csv("/Users/admin/Desktop/IRONHACK/8.WEEK/final_project/notebooks/datos/jeans/df8.csv")
df3 = pd.read_csv("/Users/admin/Desktop/IRONHACK/8.WEEK/final_project/notebooks/datos/jeans/df9.csv")

jean = pd.concat([df1, df2, df3], axis=0)

jean.drop(columns=['Unnamed: 0','desciption'], inplace=True)

# Aplicar la función a todo el DataFrame
jean = jean.applymap(remove_accents)

jean.reset_index(inplace = True)

In [44]:
jean.shape

(400, 4)

In [45]:
jean = jean.dropna(subset=['product_name'])

In [46]:
jean.shape

(387, 4)

### Clean and create new columns

- Create column made_in with the origin of the clothes.
- Create column composition of clothes.
- Create column recycled or not and wich material is recycled. 
- Create column for OCS & RCS. 
- Clean price, estandar, product_name & remove spaces, accents, symbols, quotes, etc. 

#### Column product_name: 

In [47]:
def convertir_formato(texto):
    return texto.lower().title()

In [48]:
jean["product_name"] = jean["product_name"].apply(convertir_formato)

In [49]:
jean["product_name"] = jean["product_name"].str.strip() #delete null space infront of and behind each row
jean["product_name"] = jean["product_name"].str.lstrip() #delete null space left hand

In [50]:
# Alinea el texto a la izquierda y establece una anchura fija
jean = jean.applymap(lambda x: f'{x:<50}' if isinstance(x, str) else x)

In [51]:
jean.head()

,index,product_name,price,description
0,0,Jeans Zw Collection Bootcut Tiro Alto Cropped ...,"35,95 EUR ...",['Trabajamos con programas de seguimiento para...
3,3,Jeans Z1975 Rectos Tiro Alto ...,"25,95 EUR ...",['Trabajamos con programas de seguimiento para...
4,4,Jeans Z1975 Flare Tiro Medio ...,"25,95 EUR ...",['Trabajamos con programas de seguimiento para...
5,5,Jeans Z1975 Rectos Tiro Alto ...,"25,95 EUR ...",['Trabajamos con programas de seguimiento para...
6,6,Jeans Zw Collection Bootcut Tiro Alto Cropped ...,"29,95 EUR ...",['Trabajamos con programas de seguimiento para...


#### Create regex pattern to simplify the analysis

In [52]:
patt_jean = { r"[Mm]om" : "Jean Mom",
                r"[Rr]ectos" : "Jean Rectos",
                r"[Cc]ollection" : "Jean ZW Collection",
                r"[Ss]kinny" : "Jean Skinny", 
                r"[Ww]ide" : "Jean Wide Leg" 
              }

In [53]:
def categorize_activities(text, patt_jean):
    if isinstance(text, str):  # Check if the value is a text string
        for pattern, category in patt_jean.items():
            if re.search(pattern, text):
                return category
    return "Jean Temporada"  # If it does not match any category, "Other" is returned.

In [54]:
# Clear invalid values in the "activity" column
jean["product_name"] = jean["product_name"].fillna("empty")  # Fill NaN values with an empty string
jean["product_name"] = jean["product_name"].astype(str)  # Convert all values to text strings
# Apply the function to categorize activities
jean["product_name"] = jean["product_name"].apply(lambda x: categorize_activities(x, patt_jean))
jean["product_name"].unique()

array(['Jean ZW Collection', 'Jean Rectos', 'Jean Temporada',
       'Jean Wide Leg', 'Jean Skinny', 'Jean Mom'], dtype=object)

In [55]:
jean["product_name"].value_counts()

product_name
Jean Temporada        124
Jean Rectos            91
Jean ZW Collection     69
Jean Wide Leg          63
Jean Mom               23
Jean Skinny            17
Name: count, dtype: int64

#### Column made in:

In [56]:
#Function for extract made in
def extract_origin(description):
    match = re.search(r'hecho\s+en\s+(\w+)', description, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return None

In [57]:
# Apply the function to the column description and create a new column called made_in
jean['made_in'] = jean['description'].apply(extract_origin)

In [58]:
jean['made_in'].value_counts()

made_in
Turquia       167
Bangladesh    122
Pakistan       61
Egipto         19
Tunez          16
Camboya         1
Marruecos       1
Name: count, dtype: int64

#### Column estandar:

In [59]:
# Regex pattern
patron_regex = re.compile(r'(?:estandares\s(.*))', re.IGNORECASE)

In [60]:
#Function to extract standar
def extract_estandar(description):
    # Desempaquetar la cadena de texto de la lista
    description = ast.literal_eval(description) #convertir un string a lista
    match = patron_regex.search(" ".join(description[0:1]))
    #print(" ".join(description[0:1]))
    #print(match)
    if match:
        return match.group(1)
    else:
        return None

In [61]:
# Apply the function to the column description and create a new column called made_in
jean['estandar'] = jean['description'].apply(extract_estandar)

In [62]:
def check_create(text):
    palabras_clave = ['estandar', 'sociales', 'medioambientales', 'seguridad y salud']
    for palabra in palabras_clave:
        if palabra in text:
            return True
    return False

In [63]:
jean['standard_soc_ma'] = jean['estandar'].apply(check_create)

In [64]:
jean['standard_soc_ma'].value_counts()

standard_soc_ma
True    387
Name: count, dtype: int64

In [65]:
jean.drop(columns=['estandar'], inplace=True)

#### Column composition:

In [66]:
#Function to extract composition
def extract_composition_materiales_4(df):
    
    df["comprobacion"] = np.nan
    
    for idn in df.index:
        
        description = df.loc[idn, "description"]
    
        if description != "[]":
            # Desempaquetar la cadena de texto de la lista
            res = description.strip('][').split(', ')
            s1 = res[2:4][0] #Posicion donde esta ubicado en la lista
            #print(s1)
            patron = r'(\d+)%\s*([a-zA-Z]+)'
            # Buscamos todas las coincidencias en la cadena
            coincidencias = re.findall(patron, s1)
            #print(coincidencias)
        
            df["comprobacion"][idn] = coincidencias
            
            for porcentaje, material in coincidencias:
                if material in df.columns:
                    df[material][idn] = porcentaje
                else:
                    df[material]= np.nan
                    df[material][idn] = porcentaje
        else:
            continue
                
    return df

In [67]:
apply_function8 = extract_composition_materiales_4(jean)

In [68]:
jean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 387 entries, 0 to 399
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   index             387 non-null    int64 
 1   product_name      387 non-null    object
 2   price             387 non-null    object
 3   description       387 non-null    object
 4   made_in           387 non-null    object
 5   standard_soc_ma   387 non-null    bool  
 6   comprobacion      387 non-null    object
 7   algodon           387 non-null    object
 8   poliester         32 non-null     object
 9   elastano          129 non-null    object
 10  elastomultiester  25 non-null     object
 11  liocel            27 non-null     object
 12  lino              2 non-null      object
dtypes: bool(1), int64(1), object(11)
memory usage: 47.8+ KB


In [69]:
#jean.head()

In [70]:
# Crear una nueva columna para contar el número de materiales en cada lista
jean['num_materials'] = jean['comprobacion'].apply(lambda x: len(x) if isinstance(x, list) else 0)

In [71]:
# Crear una nueva columna para asignar el main material de cada producto.

jean['material'] = jean['comprobacion'].apply(
    lambda x: x[0][1] if isinstance(x, list) and len(x) > 0 and len(x[0]) > 1 else None
)

In [72]:
jean['material'].value_counts()

material
algodon      383
liocel         3
poliester      1
Name: count, dtype: int64

In [73]:
# Crear una nueva columna para asignar el material secundario de cada producto.
jean['num_material'] = jean['comprobacion'].apply(lambda x: int(x[0][0]) if len(x) > 0 else None)

In [74]:
# Crear una nueva columna para contar el % de material para ese producto
jean['material_secundario'] = jean['comprobacion'].apply(lambda x: x[1][1] if len(x) > 1 else None)

In [75]:
# Crear una nueva columna para contar el % de material para ese producto
jean['num_material_secundario'] = jean['comprobacion'].apply(lambda x: int(x[1][0]) if len(x) > 1 else None)

In [76]:
jean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 387 entries, 0 to 399
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    387 non-null    int64  
 1   product_name             387 non-null    object 
 2   price                    387 non-null    object 
 3   description              387 non-null    object 
 4   made_in                  387 non-null    object 
 5   standard_soc_ma          387 non-null    bool   
 6   comprobacion             387 non-null    object 
 7   algodon                  387 non-null    object 
 8   poliester                32 non-null     object 
 9   elastano                 129 non-null    object 
 10  elastomultiester         25 non-null     object 
 11  liocel                   27 non-null     object 
 12  lino                     2 non-null      object 
 13  num_materials            387 non-null    int64  
 14  material                 387 no

In [77]:
columns_to_keep2 = ["comprobacion", "material", "num_materials", "num_material", "material_secundario", "num_material_secundario"]

filtered_df = jean[columns_to_keep2]
#filtered_df.tail(20)

In [78]:
jean["num_materials"].value_counts()

num_materials
1    230
2     99
3     58
Name: count, dtype: int64

#### Column recycled:

In [79]:
def recycled_2(df):
    
    # Inicializar columnas adicionales para cada tipo de material
    for material in ["OCS", "RCS", "reciclado"]:
        df[material] = np.nan
    
    # Inicializar columna de comprobación
    df["comprobacion2"] = np.nan
    
    # Iterar sobre cada índice del DataFrame
    for idn in df.index:
        # Obtener la descripción de la fila actual
        description = df.loc[idn, "description"]
        
        # Lista para almacenar los materiales encontrados
        materiales_encontrados = []
        
        # Verificar si la descripción no está vacía
        if description != "[]":
            # Desempaquetar la cadena de texto de la lista
            res = description.strip('][').split(', ')
            
            # Recorrer cada elemento de la descripción
            for item in res:
                # Buscar coincidencias con el patrón en la descripción
                matches = re.findall(r"(\d+% .*?(?:OCS|RCS|reciclado|cultivo orgánico))", item)
                
                # Iterar sobre las coincidencias encontradas
                for match in matches:
                    # Extraer el porcentaje y el tipo de material
                    porcentaje, material = match.split(None, 1)
                    
                    # Limpiar el tipo de material
                    material = material.strip()
                    
                    # Convertir el porcentaje a float
                    porcentaje = float(porcentaje.strip('%'))
                    
                    # Asignar el porcentaje al tipo de material en la fila actual
                    df.at[idn, material] = porcentaje
                    
                    # Agregar el material a la lista de materiales encontrados
                    materiales_encontrados.append(material)
                    
        # Asignar la lista de materiales encontrados a la columna de comprobación
        df.at[idn, "comprobacion2"] = ', '.join(materiales_encontrados)  # Convertir la lista a cadena de texto
                
    return df

In [80]:
prueba3 = recycled_2(jean)

In [81]:
#jean.info()

In [82]:
columns_to_keep3 = ["comprobacion2", "poliester reciclado", "algodon reciclado", "algodon de cultivo organico certificado OCS"]

filtered_df = jean[columns_to_keep3]
#filtered_df.tail(20)

In [83]:
def obtener_certificacion(lista):
    if 'algodon de cultivo organico certificado OCS' in lista \
        or 'algodon de cultivo organico' in lista \
        or 'algodon organico' in lista \
        or 'algodon certificado OCS' in lista \
        or 'algodon OCS' in lista \
        or 'algodon reciclado certificado' in lista \
        or 'algodon reciclado' in lista:
        return 'OCS'
    elif 'poliamida reciclada certificada RCS' in lista \
        or "algodon reciclado" in lista \
        or 'reciclada' in lista:
        return 'RCS'
    
    else:
        return None

In [84]:
jean['certificacion'] = jean['comprobacion2'].apply(obtener_certificacion)

In [85]:
columns_to_keep3 = ["comprobacion2", "poliester reciclado","algodon reciclado","algodon de cultivo organico certificado OCS", "certificacion"]

filtered_df = jean[columns_to_keep3]
#filtered_df.tail(20)

In [86]:
jean["certificacion"].value_counts()

certificacion
OCS    279
Name: count, dtype: int64

In [87]:
def aplicar_certificaciones(df):
    # Inicializar las nuevas columnas en el DataFrame
    df['OCS'] = np.nan
    df['RCS'] = np.nan
    
    for idx in df.index:
        descripcion = df.loc[idx, 'comprobacion2']
        
        if pd.notna(descripcion):
            if "organico" in descripcion.lower():
                df.at[idx, 'OCS'] = 'OCS'
            if "reciclado" in descripcion.lower() or "reciclada" in descripcion.lower():
                df.at[idx, 'RCS'] = 'RCS'
    
    return df

In [88]:
df_result = aplicar_certificaciones(jean)

In [89]:
#jean.head()

In [90]:
# Crear una función para obtener el porcentaje correspondiente
def obtener_porcentaje(row):
    if row['certificacion'] == 'OCS':
        return row['algodon de cultivo organico certificado OCS']
    elif row['certificacion'] == 'RCS':
        return row['poliamida reciclada certificada RCS']
    else:
        return None

In [91]:
jean['porcentaje'] = jean.apply(obtener_porcentaje, axis=1)

In [92]:
columns_to_keep3 = ["comprobacion2", "poliester reciclado","algodon reciclado","algodon de cultivo organico certificado OCS", "certificacion", "porcentaje"]

filtered_df = jean[columns_to_keep3]
#filtered_df.tail(20)

In [93]:
#jean.info()

#### Column Price: 

In [94]:
def remove_currency(df):
    df["price"] = df["price"].replace("EUR", "", regex=True)
    return df

In [95]:
remove = remove_currency(jean)

#### Column certificate por Intertek 193341:

In [96]:
def add_intertek_column(df):
    pattern = re.compile(r'Certificado\s+por\s+Intertek\s+193341', re.IGNORECASE)
    df['intertek'] = df['description'].apply(lambda x: bool(pattern.search(x)))
    
    return df

In [97]:
interket = add_intertek_column(jean)

In [98]:
jean["intertek"].value_counts()

intertek
True     279
False    108
Name: count, dtype: int64

In [99]:
def etiquetar2(df):
    df['OCS'] = "None"  # Inicializar la columna OCS con None
    df['RCS'] = "None"  # Inicializar la columna RCS con None

    for index, row in df.iterrows():
        comprobacion = row['comprobacion2']
        if isinstance(comprobacion, str):
            if 'organico' in comprobacion:
                df.at[index, 'OCS'] = 'OCS'
            if 'reciclado' in comprobacion:
                df.at[index, 'RCS'] = 'RCS'
        elif isinstance(comprobacion, list):
            for item in comprobacion:
                if 'organico' in item:
                    df.at[index, 'OCS'] = 'OCS'
                if 'reciclado' in item:
                    df.at[index, 'RCS'] = 'RCS'

    return df

In [100]:
jean = etiquetar2(jean)

In [101]:
jean["OCS"].value_counts()

OCS
OCS     239
None    148
Name: count, dtype: int64

In [102]:
jean["OCS"].value_counts()

OCS
OCS     239
None    148
Name: count, dtype: int64

In [103]:
#pivot_table2.to_csv("total_cert_jeans.csv")

### EDA

In [105]:
# Contar cuántos registros tienen valores no nulos en 'OCS' y 'RCS'
ocs_cert_count = jean['OCS'].notna().sum()
rcs_cert_count = jean['RCS'].notna().sum()

# Calcular el recuento total de elementos
total_count = len(jean)

# Calcular el porcentaje de registros con certificación en 'OCS' y 'RCS'
ocs_cert_percentage = (ocs_cert_count / total_count) * 100
rcs_cert_percentage = (rcs_cert_count / total_count) * 100

# Mostrar los porcentajes
print(f"Porcentaje de registros con certificación OCS: {ocs_cert_percentage:.2f}%")
print(f"Porcentaje de registros con certificación RCS: {rcs_cert_percentage:.2f}%")

Porcentaje de registros con certificación OCS: 100.00%
Porcentaje de registros con certificación RCS: 100.00%


In [106]:
# Contar cuántos registros tienen valores no nulos en 'OCS' y 'RCS'
ocs_cert_count = jean['OCS'].notna().sum()
rcs_cert_count = jean['RCS'].notna().sum()

# Calcular el recuento total de elementos
total_count = len(jean)

# Calcular el porcentaje de registros con certificación en 'OCS' y 'RCS'
ocs_cert_percentage = (ocs_cert_count / total_count) * 100
rcs_cert_percentage = (rcs_cert_count / total_count) * 100

# Crear un DataFrame con los porcentajes
certification_percentages = pd.DataFrame({
    'Certificación': ['OCS', 'RCS'],
    'Porcentaje': [ocs_cert_percentage, rcs_cert_percentage]
})

# Guardar el DataFrame en un archivo CSV
#
certification_percentages.to_csv("certification_percentages.csv", index=False)

# Mostrar el DataFrame para verificar
certification_percentages

,Certificación,Porcentaje
0,OCS,100.0
1,RCS,100.0


In [107]:
certification_percentages.to_csv("%_certif_jeans.csv")

In [109]:
# Calcular el total de registros por producto
total_por_producto = jean.groupby('product_name').size()

# Calcular el número de registros con certificación por producto en 'OCS' y 'RCS'
ocs_cert_por_producto = jean.groupby('product_name')['OCS'].apply(lambda x: x.notna().sum())
rcs_cert_por_producto = jean.groupby('product_name')['RCS'].apply(lambda x: x.notna().sum())

# Calcular el porcentaje de certificación por producto
ocs_cert_percentage_por_producto = (ocs_cert_por_producto / total_por_producto) * 100
rcs_cert_percentage_por_producto = (rcs_cert_por_producto / total_por_producto) * 100

# Crear un DataFrame con los porcentajes por producto
certification_percentages_por_producto = pd.DataFrame({
    'Product': total_por_producto.index,
    'OCS Percentage': ocs_cert_percentage_por_producto.values,
    'RCS Percentage': rcs_cert_percentage_por_producto.values
})

# Guardar el DataFrame en un archivo CSV
certification_percentages_por_producto.to_csv("certification_percentages_por_producto.csv", index=False)

# Mostrar el DataFrame para verificar
certification_percentages_por_producto

,Product,OCS Percentage,RCS Percentage
0,Jean Mom,100.0,100.0
1,Jean Rectos,100.0,100.0
2,Jean Skinny,100.0,100.0
3,Jean Temporada,100.0,100.0
4,Jean Wide Leg,100.0,100.0
5,Jean ZW Collection,100.0,100.0


In [110]:
certification_percentages_por_producto.to_csv("jean_%_product.csv")

## FINAL JEAN DF: 

In [111]:
jean = jean.drop(columns=['description', "comprobacion", "comprobacion2"])

In [112]:
jean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 387 entries, 0 to 399
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   index                                        387 non-null    int64  
 1   product_name                                 387 non-null    object 
 2   price                                        387 non-null    object 
 3   made_in                                      387 non-null    object 
 4   standard_soc_ma                              387 non-null    bool   
 5   algodon                                      387 non-null    object 
 6   poliester                                    32 non-null     object 
 7   elastano                                     129 non-null    object 
 8   elastomultiester                             25 non-null     object 
 9   liocel                                       27 non-null     object 
 10  lino   

In [113]:
jean.rename(columns={"num_materials": "total_composition", 
                    "material": "main_txt",
                    "num_material" : "1_composition", 
                    "material_secundario" : "second_txt",
                    "num_material_secundario" : "2_composition",
                    "reciclado" : "txt_recycled",
                    "algodon de cultivo organico certificado OCS": "cotton_OCS",
                    "certificacion" : "certificacion",
                    "poliester reciclado": "rec_poliester",
                    "algodon reciclado": "rec_cotton"}, inplace=True)

In [114]:
# Función para quitar el símbolo '%'
def remove_percent(x):
    if isinstance(x, str):
        return x.replace('%', '')
    return x

In [115]:
jean = jean.applymap(remove_percent)

In [116]:
columns_to_convert = ['algodon', 'poliester', 'elastano', 
                     'elastomultiester', 'liocel', 'lino']

# Convertir las columnas especificadas a numérico
for column in columns_to_convert:
    jean[column] = pd.to_numeric(jean[column], errors='coerce')

In [117]:
jean['price'] = pd.to_numeric(jean['price'].str.replace(',', '.'), errors='coerce')

In [118]:
#jean.info()

In [119]:
columns1 = ['algodon', 'poliester', 'elastano', 'elastomultiester', 'liocel', 'lino', '1_composition', '2_composition', 'cotton_OCS', 'rec_poliester',
                     'rec_cotton', 'porcentaje']

jean[columns1] = jean[columns1].apply(lambda x: x / 100)

In [120]:
# Calcular el total de certificaciones OCS y el total de algodón en todas las prendas
total_OCS = jean['OCS'].eq('OCS').sum()
total_algodon = jean['algodon'].sum()

# Calcular el porcentaje de OCS en función del total de algodón
porcentaje_OCS_total = (total_OCS / total_algodon) * 100

In [121]:
#jean.to_csv("jean_ok.csv", index=True)